In [ ]:
# Webscraping Script for ArtPrice

In [52]:
# Import Necessary Libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup
import time

import pandas as pd
import time
import os
import requests
import re
import string


In [91]:
driver.quit()

In [92]:
# Setup Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
# options.add_argument('--headless=new')

# Initialize Chrome driver
driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()

In [ ]:
# Login to ArtPrice
def login_to_artprice(driver, email, password):
    
    driver.get("https://www.artprice.com")
    time.sleep(5)
    
    # Accept Cookies
    try:
        accept_cookies = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.sln-accept-cookies"))
        )
        accept_cookies.click()
        print("Accepted cookies.")
    except Exception:
        print("No cookie banner or already accepted.")
    time.sleep(7)

    # Click login button
    try:
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a.e2e-login-btn"))
        )
        login_button.click()
    except Exception as e:
        print("Couldn't find login button:", e)
        return False
    
    time.sleep(7)
    
    # Fill login form
    try:
        login_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "login"))
        )
        password_input = driver.find_element(By.ID, "pass")
        
        login_input.clear()
        login_input.send_keys(email)
        password_input.clear()
        password_input.send_keys(password)

        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.e2e-login-submit-btn"))).click()
        time.sleep(5)

        print("Logged in successfully")
        return True

    except Exception as e:
        print("Login failed:", e)
        return False



In [87]:
# Setup driver
driver = webdriver.Chrome(options=chrome_options)

login_to_artprice(driver, "artauctionproject57@gmail.com", "Artauctionproject2025!")



Accepted cookies.
Logged in successfully


True

In [11]:
# Sprase URLs of each artwork per page
def get_all_artwork_links(driver, start_url):
    all_links = []
    driver.get(start_url)

    while True:
        # Wait for artwork grid to load
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "lot-container")))

        # Parse page content
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Extract artwork links
        for container in soup.find_all("div", class_="lot-container"):
            a_tag = container.find("a", class_="sln_lot_show")
            if a_tag:
                href = a_tag.get("href")
                full_url = "https://www.artprice.com" + href
                all_links.append(full_url)

        print(f"Scraped {len(all_links)} artworks so far...")

        # Try to click the "Next" button
        try:
            next_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "li.next_page a.sln-next-page"))
            )
            next_button.click()
            time.sleep(2)  # Give time for the next page to load
        except:
            print("No more pages or 'Next' button not found.")
            break

    return all_links


In [8]:
# Test Function
start_url = "https://www.artprice.com/artist/15079/wassily-kandinsky/lots/pasts?p=1"
all_artwork_links = get_all_artwork_links(driver, start_url)

print(f"Total artworks scraped: {len(all_artwork_links)}")


Scraped 30 artworks so far...
Scraped 60 artworks so far...
Scraped 90 artworks so far...
Scraped 120 artworks so far...
Scraped 150 artworks so far...
Scraped 180 artworks so far...
No more pages or 'Next' button not found.
Total artworks scraped: 180


In [100]:
# Scraping a Single Artwork
def scrape_artwork_detail(driver, url):

    driver.get(url)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # -- Artist --
    artist_tag = soup.select_one("div.artist a.invisiblelink")
    artist = artist_tag.text.strip() if artist_tag else None

    # -- Title --
    title_tag = soup.select_one("div.lot-title h1")
    title = title_tag.text.strip() if title_tag else None

    # -- Lot Number --
    lot_tag = soup.find("div", class_="block marg marg-t-5")
    lot_number = lot_tag.text.strip() if lot_tag else None

    # -- Description (h2/h3/h4 inside .description) --
    description_tag = soup.find("div", class_="description")
    if description_tag:
        desc_parts = [tag.text.strip() for tag in description_tag.find_all(["h2", "h3", "h4"])]
        description = " | ".join(desc_parts)
    else:
        description = None

    # -- Prices --
    final_price = starting_price = estimate = price_incl_premium = None
    prices_tag = soup.find("div", class_="prices")
    if prices_tag:
        # Final hammer price
        hammer = prices_tag.find("div", class_="price")
        if hammer and "Hammer price" in hammer.text:
            span = hammer.find("span", attrs={"ng-show": re.compile("currency.*eur")})
            final_price = span.text.strip() if span else None

        # Price incl. premium
        premium = prices_tag.find("div", class_="price-tax")
        if premium:
            span = premium.find("span", attrs={"ng-show": re.compile("currency.*eur")})
            price_incl_premium = span.text.strip() if span else None

        # Starting price
        start = prices_tag.find("div", class_="price-start")
        if start:
            span = start.find("span", attrs={"ng-show": re.compile("currency.*eur")})
            starting_price = span.text.strip() if span else None

        # Estimate
        estimate_spans = prices_tag.select("div.price-estim span[ng-show*='currency']")
        if estimate_spans:
            estimate = " - ".join([s.text.strip() for s in estimate_spans if "eur" in s.get("ng-show", "")])

    # -- Sale Info --
    sale_title = sale_date = auction_house = location = None
    sale_tag = soup.find("div", class_="sale")
    
    if sale_tag:
        sale_title_tag = sale_tag.select_one("div.strong.block i")
        sale_title = sale_title_tag.text.strip() if sale_title_tag else None

        sale_date_tag = sale_tag.select_one("div.block.sale-date")
        sale_date = sale_date_tag.text.strip() if sale_date_tag else None

        auction_house_tag = sale_tag.select_one("div.block.strong div")
        auction_house = auction_house_tag.text.strip() if auction_house_tag else None

        # Get last meaningful block for location
        location = "EDIT"

    # -- Additional Details --
    additional_details = {}
    datas_tag = soup.select_one("div.sale div.datas")
    if datas_tag:
        for block in datas_tag.find_all("div", class_="block"):
            head_span = block.find("span", class_="head")
            if head_span:
                label = head_span.text.strip().rstrip(":")
                value = block.get_text(separator=" ", strip=True).replace(f"{label}:", "").strip()
                additional_details[label] = value
            else:
                # Try long content (like exhibition blocks)
                content = block.get_text(separator=" ", strip=True)
                if content:
                    key = f"Note {len(additional_details)+1}"
                    additional_details[key] = content
    
    # Image URL
    image_url = None
    img_tag = soup.select_one("div.img-tag img")
    if img_tag and img_tag.get("src"):
        image_url = img_tag["src"]

    return {
        "URL": url,
        "Artist": artist,
        "Title": title,
        "Lot Number": lot_number,
        "Description": description,
        "Final Price (EUR)": final_price,
        "Price Incl. Premium (EUR)": price_incl_premium,
        "Starting Price (EUR)": starting_price,
        "Estimate (EUR)": estimate,
        "Sale Title": sale_title,
        "Sale Date": sale_date,
        "Auction House": auction_house,
        "Location": location,
        "Image URL": image_url,
        "Additional Details": additional_details
    }

def flatten_artwork_data(records):
    flat_records = []

    for record in records:
        flat = record.copy()
        details = flat.pop("Additional Details", {})  # Remove nested dict and store keys
        for key, value in details.items():
            flat[f"Detail - {key}"] = value  # Prefix to avoid column name clashes
        flat_records.append(flat)

    return flat_records


In [101]:
test_url = "https://www.artprice.com/artist/15079/wassily-kandinsky/drawing-watercolor/35952741/friedlich?p=2"
data = scrape_artwork_detail(driver, test_url)
pd.DataFrame([data])

flat_data = flatten_artwork_data([data])
df = pd.DataFrame(flat_data)
display(df.head())



,URL,Artist,Title,Lot Number,Description,Final Price (EUR),Price Incl. Premium (EUR),Starting Price (EUR),Estimate (EUR),Sale Title,Sale Date,Auction House,Location,Image URL,Detail - Note 1,Detail - Note 2,Detail - Literature,Detail - Provenance,Detail - Exhibition,Detail - Note 6
0,https://www.artprice.com/artist/15079/wassily-...,Wassily KANDINSKY (1866-1944),Friedlich\n (1930),Lot #\n 8,"Drawing-Watercolor | Watercolour, ink & pen/pa...","€ 620,000","€ 787,400",None,"€ 300,000 - € 500,000",Evening Sale,06 dec 2024,Ketterer Kunst GmbH,EDIT,https://imgprivate2.artprice.com/lots/MzMzNzQw...,Details,Monog. dated\n \n lo...,"Literature : Endicott Barnett, II, 987","Provenance : Paul Klee Collection (1879-1940),...",Exhibition : Memorial exhibition of Wassily Ka...,Illustrated on page 19 of the catalog


In [ ]:
# All Artist Names and URLs
BASE_URL = "https://www.artprice.com"

def scrape_artist_links(driver, letter):
    links = []
    url = f"{BASE_URL}/artists/all/{letter}"
    print(f"Starting {url}")
    driver.get(url)

    try:
        while True:
            time.sleep(2)
            artists = driver.find_elements(By.CSS_SELECTOR, "div.col-xs-6.artist a")
            for a in artists:
                href = a.get_attribute("href")
                if href:
                    links.append(href)

            # Try clicking next
            try:
                next_btn = driver.find_element(By.CSS_SELECTOR, "a.next")
                driver.execute_script("arguments[0].scrollIntoView(true);", next_btn)
                driver.execute_script("arguments[0].click();", next_btn)
            except NoSuchElementException:
                break
    except Exception as e:
        print(f"Error during scraping {letter}: {e}")

    return links

# --- Create driver once and reuse ---
options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # Optional: run in headless mode
# options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"  

driver = webdriver.Chrome(options=options)

# Main scraping loop
all_links = []
try:
    for letter in string.ascii_uppercase:
        print(f"\nScraping letter: {letter}")
        letter_links = scrape_artist_links(driver, letter)
        all_links.extend(letter_links)
        print(f" {len(letter_links)} links found for letter {letter}")
finally:
    driver.quit()




Scraping letter: A
Starting https://www.artprice.com/artists/all/A
✓ 24314 links found for letter A

Scraping letter: B
Starting https://www.artprice.com/artists/all/B


KeyboardInterrupt: 

In [ ]:
# Filter Artists with > 10 Artworks
def get_artist_info_if_has_enough_works(driver, artist_url, min_works=10):
    driver.get(artist_url)
    time.sleep(2)

    try:
        # Find the div that contains "Find prices (X)"
        elem = driver.find_element(By.XPATH, "//div[contains(text(), 'Find prices')]")
        match = re.search(r'Find prices\s*\((\d+)\)', elem.text)
        if match:
            count = int(match.group(1))
            if count > min_works:
                # Get artist name from the page title or h1
                try:
                    artist_name = driver.find_element(By.TAG_NAME, "h1").text
                except NoSuchElementException:
                    artist_name = artist_url.split("/")[-1]
                return {"name": artist_name, "url": artist_url, "count": count}
    except NoSuchElementException:
        pass  # If the div isn't found, skip this artist

    return None

In [ ]:
# Test Function
all_artist_links = all_links[20:25]

qualified_artists = []

driver = webdriver.Chrome()

try:
    for idx, artist_url in enumerate(all_artist_links):
        print(f"Checking {idx+1}/{len(all_artist_links)}: {artist_url}")
        result = get_artist_info_if_has_enough_works(driver, artist_url)
        if result:
            qualified_artists.append(result)
            print(f"{result['name']} has {result['count']} works")

        time.sleep(1)

finally:
    driver.quit()

# Print
print(f"\nFound {len(qualified_artists)} artists with > 10 works")


Checking 1/5: https://www.artprice.com/artist/145871/olga-harriet-aae
Checking 2/5: https://www.artprice.com/artist/959850/simon-aaen
Checking 3/5: https://www.artprice.com/artist/496930/erik-aaes
Checking 4/5: https://www.artprice.com/artist/607173/edwin-aafjes
Checking 5/5: https://www.artprice.com/artist/4/carl-frederik-aagaard
✓ Carl Frederik AAGAARD has 440 works

Found 1 artists with > 10 works


In [ ]:
print(all_links[:5])

['https://www.artprice.com/artist/945507/petrus-vander-aa', 'https://www.artprice.com/artist/392776/mario-koninck-de', 'https://www.artprice.com/artist/775143/hubertus-aab', 'https://www.artprice.com/artist/571839/ines-c-aab-tamsen', 'https://www.artprice.com/artist/396299/gunhild-aaberg']
